In [1]:
import sqlite3
conn = sqlite3.connect("nominations.db")

In [2]:
schema = conn.execute('pragma table_info(nominations);').fetchall()
schema

[(0, 'Year', 'INTEGER', 0, None, 0),
 (1, 'Category', 'TEXT', 0, None, 0),
 (2, 'Nominee', 'TEXT', 0, None, 0),
 (3, 'Won', 'INTEGER', 0, None, 0),
 (4, 'Movie', 'TEXT', 0, None, 0),
 (5, 'Character', 'TEXT', 0, None, 0)]

In [3]:
first_ten = conn.execute('select * from nominations limit 10;').fetchall()
for i in first_ten:
    print(i)

(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting Role', 'Geoffrey Rush', 0, "The King's Speech", 'Lionel Logue')


In [4]:
create_ceremonies = 'create table ceremonies (id integer primary key, Year integer, Host text);'
conn.execute(create_ceremonies)
                

OperationalError: table ceremonies already exists

In [ ]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]
insert_query = "INSERT INTO ceremonies (Year, Host) VALUES (?,?);"

In [5]:
conn.executemany(insert_query, years_hosts)

NameError: name 'insert_query' is not defined

In [ ]:
conn.execute('select * from ceremonies limit 10').fetchall()

In [ ]:
conn.execute('pragma table_info(ceremonies)').fetchall()

In [ ]:
conn.execute('pragma foreign_keys = ON;')

In [ ]:
conn.execute('''
            create table nominations_two(
            id integer primary key,
            category text,
            nominee text,
            movie text,
            character text,
            won text,
            ceremony_id integer,
            foreign key (ceremony_id) references ceremonies(id)
            );''')

In [ ]:
joined_nominations = conn.execute('''select nominations.category,nominations.nominee,nominations.movie,nominations.character,nominations.won,ceremonies.id from nominations
                                inner join ceremonies on
                                nominations.year == ceremonies.year;''').fetchall()

In [ ]:
insert_nominations_two = '''insert into nominations_two (category, nominee, movie, character, won, ceremony_id)
values (?,?,?,?,?,?);'''

In [ ]:
conn.executemany(insert_nominations_two, joined_nominations)

In [ ]:
conn.execute('select * from nominations_two limit 5;').fetchall()

In [ ]:
drop_nominations = "drop table nominations;"
conn.execute(drop_nominations)

rename_nominations_two = "alter table nominations_two rename to nominations;"
conn.execute(rename_nominations_two)

In [ ]:
create_movies = "create table movies (id integer primary key, movie text);"
conn.execute(create_movies)

In [ ]:
create_actors = "create table actors (id integer primary key, actor text);"
conn.execute(create_actors)

In [6]:
create_movies_actors = "create table movies_actors (id integer primary key, foreign key movie_id references movies(id))"

In [8]:
insert_movies = "insert into movies (movie) select distinct movie from nominations_2;"
insert_actors = "insert into actors (actor) select distinct nominee from nominations_2;"
conn.execute(insert_movies)
conn.execute(insert_actors)

In [10]:
conn.execute('select * from movies limit 5;').fetchall()

[(1, 'Biutiful'),
 (2, 'True Grit'),
 (3, 'The Social Network'),
 (4, "The King's Speech"),
 (5, '127 Hours')]

In [11]:
conn.execute('select * from actors limit 5;').fetchall()

[(1, 'Javier Bardem'),
 (2, 'Jeff Bridges'),
 (3, 'Jesse Eisenberg'),
 (4, 'Colin Firth'),
 (5, 'James Franco')]

In [13]:
query = "select movie,nominee from nominations_2;"
movie_actor_pairs = conn.execute(query).fetchall()
join_table_insert = "insert into movies_actors (movie_id, actor_id) values ((select id from movies where movie == ?),select id from actors where actor == ?));"
conn.executemany(join_table_insert, movie_actor_pairs)

OperationalError: near "select": syntax error